# Kreisdiagramme
Kreisdiagramme eignen sich insbesondere für die Darstellung von Anteilen.

## Vorverarbeitung
Importieren Sie zunächst `pandas` und `plotly` und laden Sie den Datensatz [Disney+ Movies and TV Shows](https://www.kaggle.com/code/werador/disney-data-analysis/data).

In [ ]:
import pandas as pd
import plotly.express as px
from ipywidgets import interact

df = pd.read_csv('disney_plus_titles.csv', parse_dates=['date_added'])
df = df[['type', 'director', 'release_year']]

df

Die nachfolgende Zelle reduziert die Regisseure auf die Anzahl ihrer Filme seit 2019 und setzt den Namen derer, die lediglich an einer einzigen Produktion mitwirkten, auf `Andere`.

In [ ]:
df2 = df[df['type'] == 'Movie']
df3 = df2[df2['release_year'] >= 2019][['director']].dropna()
df4 = df3['director'].str.split(', ', expand=True).stack().reset_index(level=1, drop=True).value_counts()
df5 = pd.DataFrame({'name': df4.index, 'count': df4.values})
df5.loc[df5['count'] <= 1, 'name'] = 'Andere'

Zusätzlich ist die Aufteilung nach Film und Serie von Interesse. Zur Veranschaulichung wird noch eine dritte, künstliche Kategorie `Unbekannt` eingeführt, die so im Datensatz nicht auftaucht.

In [ ]:
df6 = pd.DataFrame({
    'type': ['Serien', 'Filme', 'Unbekannt'],
    'count': [
        len(df[df['type'] == 'TV Show']),
        len(df[df['type'] == 'Movie']),
        150
    ]
})

## Ein simples Kreisdiagramm
Kreisdiagramme werden mit der Funktion `pie` erstellt. Als erster Parameter dient das DataFrame, als `names` wird der Name der Spalte mit den anzuzeigenden Namen übergeben und als `values` der Name der Spalte mit den Werten. Ist ein Name doppelt vorhanden - wie im Beispiel *Andere* - so werden diese automatisch zusammengefasst.

In [ ]:
px.pie(df5, values='count', names='name')

Mit Hilfe des Parameters `hole` lässt sich dieser Kreis in einen Donut transformieren.

In [ ]:
@interact(hole=(0, 0.95, 0.05))
def _(hole):
    return px.pie(df5, values='count', names='name', hole=hole)

## Hervorgehobene Teile
Durch den Parameter `pull` lassen sich einzelne Kreisteile aus der Mitte herausziehen und somit hervorheben.

In [ ]:
@interact(pull=(0, 1, 0.05))
def _(pull):
    fig = px.pie(df6, names='type', values='count')
    fig.update_traces(pull=[pull, 0, 0])
    return fig